In [1]:
import pickle
import time
import csv
from selenium import webdriver 
from selenium.webdriver.common.by import By

import pandas as pd 

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
options.add_argument('user-agent=' + user_agent)
# options.add_argument('headless')

driver = webdriver.Chrome('C:/final/final_sh/chromedriver.exe', options=options)

C:\Users\admin\AppData\Local\Temp\ipykernel_2932\1785440232.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/final/final_sh/chromedriver.exe', options=options)


In [5]:
# url_crawler

while 1 : 
    # 데이터 저장 (dataToCsv)
    f = open('keyword_url.csv', 'a', encoding='utf-8-sig', newline='')
    writer = csv.writer(f)
    
    with open('keyword_list4.pkl', 'rb') as f2:
        keyword_list4 = pickle.load(f2)
        
    keyword = keyword_list4[0]
    
    url = 'https://search.naver.com/search.naver?where=blog&query=부산여행 ' + keyword + '&sm=tab_opt&nso=so:r,p:from20220101to20221031'

    driver.get(url)
    time.sleep(3)

    n = 35
    i = 0
    while i < n:
        driver.execute_script("window.scrollTo(0, 99999999)")
        time.sleep(1)
        i = i+1
        
    # 블로그 글 url들 수집
    url_list = []
    title_list = []

    # URL_raw 크롤링 시작
    try:
        article_raw = driver.find_elements(By.CSS_SELECTOR, ".api_txt_lines.total_tit")
        article_raw[0]

        # 크롤링한 url 정제 시작
        for article in article_raw:
            url = article.get_attribute('href')   
            writer.writerow([keyword, url])

    except : pass
   
    del keyword_list4[0]   
    with open('keyword_list4.pkl', 'wb') as f3:
            pickle.dump(keyword_list4, f3)

IndexError: list index out of range

In [2]:
# blog_content_crawler

with open('start_iloc.pkl', 'rb') as f2:
    start_iloc = pickle.load(f2)

f = open('blog_content.csv', 'a', encoding='utf-8-sig', newline='')
writer = csv.writer(f)

df = pd.read_csv('keyword_url.csv', names=['keyword', 'url'])
print(df)

for i in range(start_iloc, len(df['keyword'])):
    keyword = df.iloc[i,0]
    url = df.iloc[i,1]
    driver.get(url)   # 글 띄우기
    
    try: 
        # 글의 iframe 접근
        driver.switch_to.frame("mainFrame")

        # 제목 크롤링
        overlays = ".se-module.se-module-text.se-title-text"                  
        title = driver.find_element(By.CSS_SELECTOR, overlays).text
        
        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"    
        datetime = driver.find_element(By.CSS_SELECTOR, overlays).text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"
        # .se-module.se-module-text                                 
        contents = driver.find_elements(By.CSS_SELECTOR, overlays)

        content_list = []
        for content in contents:
            content_list.append(content.text)

        content_str = ' '.join(content_list)
        
        # 태그 크롤링
        overlays = ".item.pcol2.itemTagfont._setTop"
        tags = driver.find_elements(By.CSS_SELECTOR, overlays)

        tag_list = []
        for tag in tags:
            tag_list.append(tag.text)
        
        tag_str = ' '.join(tag_list)
        
        
        writer.writerow([keyword, url, title, datetime, content_str, tag_str])
        
        title, datetime, content_str, tag_str = '', '', '', ''
        
        time.sleep(1)
        
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동
    except:
        print(str(i)+' failed')  
        writer.writerow([keyword, url, '', '', '', ''])
        time.sleep(1)
        continue
    
    with open('start_iloc.pkl', 'wb') as f3:
            pickle.dump(i, f3)
    print(i)
    
f.close()



        keyword                                                url
0           해월정         https://blog.naver.com/ds5dkm/222861757227
1           해월정      https://blog.naver.com/djeon0011/222874173114
2           해월정       https://blog.naver.com/yangklee/222760710717
3           해월정      https://blog.naver.com/ymart1103/222846129569
4           해월정  https://blog.naver.com/fragmentary53/222627148345
...         ...                                                ...
15609  내원정사(부산)        https://blog.naver.com/kem8732/222636169804
15610  내원정사(부산)      https://blog.naver.com/genie9756/222841250338
15611  내원정사(부산)        https://blog.naver.com/ehyndoi/222835720501
15612  내원정사(부산)     https://blog.naver.com/okok951022/222737619983
15613  내원정사(부산)       https://blog.naver.com/ssim2568/222893489120

[15614 rows x 2 columns]
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944 failed
6945
6946
6947
6948 failed
6949
6950
6951
6952
6953
6954
6955
6956
6

In [12]:
start_iloc = 0
with open('start_iloc.pkl', 'wb') as f3:
    pickle.dump(start_iloc, f3)